# Quicklook maker

This is the code helps you to plot lidar quicklooks. If you are not confident in Python, don't worry! You only need to make modifications to the statements found as follows:

#------------------------------------------------------------------------------------------------------------------------------

         Modify ONLY within here
#------------------------------------------------------------------------------------------------------------------------------

Important:
- A recurring error may be that paths are specified with '\\' as the folder separator, while the Python language uses '\\\\' (double).
- It is also recommended to run the cells one by one, and not to re-run a cell while it is executing (the software may hang). If while a cell is running, any changes are made to the code in that cell and we want to re-run it, we should wait for it to finish or restart the kernels.
- If desired, you can play with the rest of the code.

## IMPORTING LIBRARIES AND SPECIFYING PATHS

First, we need to import the necessary libraries. In case any of the modules are not found when running, install the module using:

#------------------------------------------------------------------------------------------------------------------------------

    !pip install <module_name>
#------------------------------------------------------------------------------------------------------------------------------


In [ ]:
from pathlib import Path

import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter
import xarray as xr

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.utils.plot import font_axes

%load_ext autoreload
%autoreload 2

%matplotlib inline

In the next cell, we define the directory where the lidar measurement files are located and the directory where we want to save the graphs.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
lidar_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\data\lidar')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
lidar_dir = Path(r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\02\22')
output_dir = Path(r'C:\Users\Usuario\Mi unidad\04. Proyectos\ATMO-ACCESS\curso\output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

lidar_dir.is_dir(), output_dir.is_dir() #Comprueba que ambas carpetas existen.


**TIME SERIES OF RANGE-CORRECTED SIGNAL WITH VERTICAL RESOLUTION**

We **plot the time series with vertical resolution (commonly known as 'quicklooks')** of the lidar measurement data. In our case, we represent the value of the range-corrected signal (RCS) measured in a specific channel (indicated in the graph title). The different channels are:
- 355xta (total signal at 355 nm)
- 532xca; 532xda; 532xpa; 532xpp (signals discriminating polarization at 532 nm, which will be used in another practice)
- 532xta (total signal at 532 nm) **We will work with the total component of the 532 nm channel in this practice**
- 1064xta (total signal at 1064 nm)

The function `date_quicklook()` will plot the quicklooks of the selected channels for a period specified by `ini_date` and `end_date`. In our case, we are working with a single day, so `end_date` = `ini_date`. The quicklooks will be automatically saved in the folder specified by `qlpath`, using the format used in the database of the Atmospheric Physics group at UGR. The process takes some time, and some of the comments may appear in French. The comments displayed on this tab are not relevant unless they indicate an error that interrupts the flow.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
RS_FL = lidar_dir / 'alh_1a_Prs_rs_xf_20230222.nc' #lidar file
#------------------------------------------------------------------------------------------------------------------------------

lidar = preprocess(RS_FL)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
qlpath = output_dir / 'quicklooks' #Output quicklook folder path. It is suggested to mantain the proposed path
channels = ['1064fta', '532fta'] #Channels to be used
bar_limits = {"1064fta": (0,2e8), '532fta': (0,2e7)} 
time_tuple = ('2023-02-22 11:00:00', '2023-02-22 18:00:00') #Time range to be used
range_tuple = (0, 10000.0) #Range range to be used
save_fig = False #If True, the quicklook will be saved in the output folder
dpi_fig = 300 #DPI of the quicklook
#------------------------------------------------------------------------------------------------------------------------------

#To plot a given time period or height range uncomment
#------------------------------------------------------------------------------------------------------------------------------
# lidar = lidar.sel(time=slice(*time_tuple), range=slice(*range_tuple))
#------------------------------------------------------------------------------------------------------------------------------

#Create the output folder if it does not exist
qlpath.mkdir(parents=True, exist_ok=True)

#quicklook plot
for channel_ in channels:
    if channel_ in lidar.channel.values:
        fig, ax = quicklook_xarray(lidar[f'signal_{channel_}'], is_rcs=False, scale_bounds=bar_limits[channel_])
        ax.set_ylim(0,8000)
        plt.show()
        if save_fig:
            figurepath = qlpath / f'{channel_}_quicklook.png'
            fig.savefig(figurepath.absolute().as_posix(), dpi=dpi_fig)
    else:
        print('Channel not found.')
